
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [2]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
!pip install h5py

ERROR: Operation cancelled by user
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
     |█████████████████████████▎      | 5.4 MB 537 kB/s eta 0:00:03
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 128, in resolve
    requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 473, in resolve
    state = resolution.resolve

In [3]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

/content
'=2.0.1'   sample_data
Fri Nov  5 06:38:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                        

In [4]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os
import h5py
import io
import shutil

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import tensorflow as tf

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer, GenericMask
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

RuntimeError: ignored

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### PNG to H5PY

In [ ]:
save_path = os.path.join("data/images.hdf5")
img_path_train = os.path.join(BASE_DIR,"data/train")
img_path_test = os.path.join(BASE_DIR,"data/test")
os.makedirs("data", exist_ok=True)
!ls data
if "images.hdf5" not in os.listdir("data"):
  print("producing new hdf5")
  hf = h5py.File(save_path,'a')
  train = hf.create_group("train")
  test = hf.create_group("test")
  val = hf.create_group("val")
else:
  print("removing old hdf5")
  os.remove(save_path)
  !ls data
  print("done")
  print("producing new hdf5")
  hf = h5py.File(save_path,'a')
  !ls data
  train = hf.create_group("train")
  test = hf.create_group("test")
  val = hf.create_group("val")


for img in random.sample(os.listdir(img_path_train),20):
  if img not in hf['val']:
    img_path = os.path.join(img_path_train, img)
    image_data = Image.open(img_path)
  
    val.create_dataset(img, data=np.asarray(image_data))

for img in os.listdir(img_path_train):
  if img not in hf['train'] and img not in hf['val']:
    img_path = os.path.join(img_path_train, img)
    image_data = Image.open(img_path)
    
    train.create_dataset(img, data=np.asarray(image_data))

for img in os.listdir(img_path_test):
  if img not in hf['test']:
    img_path = os.path.join(img_path_test, img)
    image_data = Image.open(img_path)

    test.create_dataset(img, data=np.asarray(image_data))

print("done")
!ls data
hf.close()

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
#hf = h5py.File(save_path, 'r')
#hf.close()
def get_detection_data(set_name):
  data_dirs = '{}/data'.format(BASE_DIR)
  dataset = []
  tracker = {}
  tracker_i = 0

  if set_name == "train" or set_name == "val":
    json_file = os.path.join(data_dirs, "train.json")
    with open(json_file) as f:
      imgs = json.load(f)
    data_dirs = os.path.join(data_dirs,"train")

  if set_name == "test":
    data_dirs = os.path.join(data_dirs,"test")
    imgs = os.listdir(data_dirs)

  hf = h5py.File(save_path, 'r')

  for idx, v in enumerate(imgs):
    record = {}
    objs = []

    if set_name == "train" or set_name == "val":
      index = v["file_name"]
      filename = os.path.join(data_dirs,v["file_name"])
      id = v['image_id']
      obj = {
        "bbox": v["bbox"],
        "bbox_mode": BoxMode.XYWH_ABS,
        "segmentation": v["segmentation"],
        "category_id": 0,
        "iscrowd": v["iscrowd"],}
    elif set_name == "test":
      id = idx
      index = v
      filename = os.path.join(data_dirs,v)
      obj = {"category_id": 0,}
    if index not in tracker:
      if index in hf[set_name]:
        size = list(hf[set_name][index].shape)
        height = size[0]
        width = size[1]
      else:
        continue

      tracker[index] = tracker_i
      tracker_i = tracker_i + 1

      #common
      record["file_name"] = filename  #full path of image files.
      record["image_id"] = id  #test set does not have json file, so we use idx instead of actual image_id.
      record["height"] = height
      record["width"] = width

      objs.append(obj)
      record["annotations"] = objs

      #instance detection/segmentation
      dataset.append(record)
    else:
      dataset[tracker[index]]["annotations"].append(obj)

  hf.close()  
  return dataset

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
#plane_metadata = {}
for d in ["train", "test", "val"]:
  DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
  MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("data_detection_train")


In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
#hf = h5py.File(save_path, 'r')
#data = list(hf['test']['P0161.png'].shape)
#print(data)
#hf.close()

dataset_dicts = get_detection_data("train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plane_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
#COCO-Detection/retinanet_R_101_FPN_3x.yaml RETINANET
#COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml FPN
#COCO-Detection/rpn_R_50_FPN_1x.yaml RPN
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ("data_detection_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 # 2 = 1 58%
cfg.SOLVER.BASE_LR = 0.025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = [200,]#[300,]        # decay learning rate
cfg.SOLVER.GAMMA = 0.1
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 64
#cfg.INPUT.MIN_SIZE_TRAIN = 1000
#cfg.INPUT.MIN_SIZE_TEST = 1000
cfg.INPUT.MAX_SIZE_TRAIN = 3000 # 8000=6000 = 4000 58%
cfg.INPUT.MAX_SIZE_TEST = 3000

#cfg.TEST.DETECTIONS_PER_IMAGE = 1000

### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/CMPT_CV_lab3/output

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.6 #for RETINA
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 5):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
!ls drive/MyDrive/CMPT_CV_lab3/output
evaluator = COCOEvaluator("data_detection_val", output_dir = cfg.OUTPUT_DIR)
train_loader = build_detection_test_loader(cfg, "data_detection_val")
print(inference_on_dataset(predictor.model, train_loader, evaluator))

### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''
#chainging hyperparameters like lr(0.0002~0.02 were tested), batch size(512 256 128 64 are tested), baseline models(fast R-CNN, RetinaNet, and RPN are tested) improved performances.

## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''

def get_instance_sample(data, idx, set_name, img=None):
  obj_mask = GenericMask(data['annotations'][idx[1]]['segmentation'],data['height'],data['width']).mask
  #obj_img = cv2.imread(data['file_name'])
  #print(obj_img.shape)
  filename = data["file_name"][-9:]
  hf = h5py.File(save_path, 'r')
  obj_img = np.array(hf[set_name][filename]).astype("uint8")
  x = int(data['annotations'][idx[1]]['bbox'][0])
  y = int(data['annotations'][idx[1]]['bbox'][1])
  w = int(data['annotations'][idx[1]]['bbox'][2])
  h = int(data['annotations'][idx[1]]['bbox'][3])
  obj_mask = obj_mask[y:y+h, x:x+w]
  obj_img = obj_img[y:y+h, x:x+w]
  hf.close()
  return obj_img, obj_mask

In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    
    img, mask = get_instance_sample(data, idx, self.set_name)
    img = cv2.resize(img, dsize = (128,128))
    mask = cv2.resize(mask, dsize = (128,128))
    mask = mask[np.newaxis, :, : ]
    if len(img.shape) == 2:
      img = cv2.merge((img,img,img))
    
    img, mask = self.numpy_to_tensor(img, mask)
    

    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        
        self.input_conv = conv(3, 4) #conv = conv2d + BN + ReLu
        self.down = down(4, 8) #conv + pool
        ############################################################
        self.en_conv1 = conv(8, 8)
        self.down1 = down(8,16)
        ##########################################################
        self.en_conv21 = conv(16, 16)
        self.en_conv22 = conv(16, 16)
        self.down2 = down(16,32)
        ########################################################
        self.en_conv31 = conv(32, 32)
        self.en_conv32 = conv(32, 32)
        self.down3 = down(32,64)
        ########################################################
        self.en_conv41 = conv(64, 64)
        self.en_conv42 = conv(64, 64)
        self.down4 = down(64,128)
        
        # Decoder
        self.up4 = up(128, 64)
        self.de_conv41 = conv(64, 64)
        self.de_conv42 = conv(64, 64)
        ###############################################################
        self.up3 = up(64, 32)
        self.de_conv31 = conv(32, 32)
        self.de_conv32 = conv(32, 32)
        ###################################################################
        self.up2 = up(32, 16)
        self.de_conv21 = conv(16, 16)
        self.de_conv22 = conv(16, 16)
        ####################################################################
        self.up1 = up(16, 8)
        self.de_conv1 = conv(8, 8)
        #######################################################################
        self.up = up(8, 4) ##conv + unpool
        self.output_conv = conv(4, 1, False) # ReLu activation is removed to keep the logits for the loss function
        

    def forward(self, input):
      y = self.input_conv(input)
      y = self.down(y)
      y  = self.en_conv1(y)
      y = self.down1(y)
      y  = self.en_conv21(y)
      y  = self.en_conv22(y)
      y = self.down2(y)
      y  = self.en_conv31(y)
      y  = self.en_conv32(y)
      y = self.down3(y)
      y  = self.en_conv41(y)
      y  = self.en_conv42(y)
      y = self.down4(y)
      y = self.up4(y)
      y = self.de_conv41(y)
      y = self.de_conv42(y)
      y = self.up3(y)
      y = self.de_conv31(y)
      y = self.de_conv32(y)
      y = self.up2(y)
      y = self.de_conv21(y)
      y = self.de_conv22(y)
      y = self.up1(y)
      y = self.de_conv1(y)
      y = self.up(y)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
def dice_loss(predict, gt):
  m = nn.Sigmoid()
  predict_sig = m(predict)
  bat = predict.shape[0]
  mul = predict_sig * gt
  mul = mul.sum(2)
  mul = mul.sum(2)
  numerator = 2*mul + 1
  predict_sum = predict_sig.sum(2)
  predict_sum = predict_sum.sum(2)
  gt_sum = gt.sum(2)
  gt_sum = gt_sum.sum(2)
  denominator = predict_sum + gt_sum + 1
  dc = 1 - (numerator/denominator)
  dc = (dc.sum(0)/bat)[0]
  return dc

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 15 #best at 10, trying 15
batch_size = 4
learning_rate = 0.007
weight_decay = 0.00001

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    loss1 = crit(pred, mask)
    loss2 = dice_loss(pred,mask)
    loss = loss1 + loss2
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

m = nn.Sigmoid()
total_iou = 0
pic_num = 0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    mask = torch.unsqueeze(mask,1)
    pred = model(img)
    pred = m(pred)
    pred = pred.cpu().detach().numpy()
    mask = mask.cpu().detach().numpy()
    bat_pred = pred.shape[0]
    bat_mask = mask.shape[0]
    pred = pred.reshape(bat_pred,128,128,1)
    mask = mask.reshape(bat_mask,128,128,1)
    pred[pred < 0.6] = 0
    pred[pred >= 0.6] = 1
    if mask.shape[0] == pred.shape[0]:
      for i in range(mask.shape[0]):
        pic_num = pic_num + 1
        intersection = np.logical_and(pred[i], mask[i])
        union = np.logical_or(pred[i], mask[i])
        total_iou = total_iou + (np.sum(intersection) / np.sum(union))
    else:
      print("mask and input has different size")


    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    
mean_iou = total_iou / pic_num    

print("\n #images: {}, Mean IoU: {}".format(pic_num, mean_iou))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
hf = h5py.File(save_path, 'r')
transform = transforms.Compose([transforms.ToTensor(),])
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):
    #img = cv2.imread(d["file_name"])
    #print(img.shape)
    filename = d["file_name"][-9:]
    img = np.array(hf["test"][filename]).astype("uint8")
    if len(img.shape) == 2:
      img = cv2.merge((img,img,img))
    outputs = predictor(img)
    bbox = np.array(np.array(np.array(outputs['instances'].pred_boxes[0]))[0].to("cpu"))
    #bbox = outputs['instances'].pred_boxes
    #for box in bbox:
    #  print(int(box[0]))
    #print(len(np.array(outputs['instances'].pred_boxes)))
    #breakline
    x = int(bbox[0])
    y = int(bbox[1])
    w = int(bbox[2])
    h = int(bbox[3])
    img = img[y:h, x:w]
    img = cv2.resize(img,dsize=(128,128))
    cv2_imshow(img)
    img = transform(img)
    img = img[np.newaxis, :, : ]
    #img = torch.tensor(img, dtype=torch.float)
    img = img.cuda()
    pred = model(img)
    pred = m(pred)
    pred = pred.cpu().detach().numpy()
    pred = pred.reshape(pred.shape[2],pred.shape[3],pred.shape[1])
    pred[pred < 0.6] = 0
    pred[pred >= 0.6] = 1
    pred = pred*255
    cv2_imshow(pred)
hf.close()

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''
def get_prediction_mask(data, set_name):
  m = nn.Sigmoid()
  hf = h5py.File(save_path, 'r')
  filename = data["file_name"][-9:]

  img = np.array(hf[set_name][filename]).astype("uint8")
  if len(img.shape) == 2:
    img = cv2.merge((img,img,img))

  
  pred_mask = torch.zeros(data['height'], data['width']).cuda()
  gt_mask = torch.zeros(1,data['height'], data['width']).cuda()
  outputs = predictor(img)
  pred_bbox = outputs['instances'].pred_boxes

  for ind, box in enumerate(pred_bbox):
    x = int(box[0])
    y = int(box[1])
    w = int(box[2])
    h = int(box[3])
    seg = img[y:h, x:w]
    seg = transform(seg)
    seg = seg.unsqueeze(0)
    seg = F.interpolate(seg, size=(128,128))
    seg = seg.squeeze(0)
    seg =  seg.reshape(1,seg.shape[0], seg.shape[1],seg.shape[2])
    seg = seg.cuda()
    seg_mask = model(seg)
    seg_mask = m(seg_mask)
    seg_mask = torch.where(seg_mask<0.6, 0., float(ind+1))
    seg_mask = F.interpolate(seg_mask, size=(h-y,w-x))
    pred_mask[y:h, x:w] = seg_mask

  #img = transform(img)
  pred_mask = pred_mask.unsqueeze(0)

  
  if set_name == 'train' or set_name == 'val':
    for i in range (len(data['annotations'])):
      mask = GenericMask(data['annotations'][i]['segmentation'],data['height'],data['width']).mask
      mask =  mask.reshape(1,mask.shape[0], mask.shape[1])
      mask = torch.tensor(mask, dtype=torch.float).cuda()
      gt_mask = gt_mask + mask

  hf.close()
  return img, gt_mask, pred_mask # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('val'))
for i in tqdm(range(8,9), position=0, leave=True):
  sample = my_data_list[i]
  img, gt_mask, pred_mask = get_prediction_mask(sample, 'val')
  pred_mask = pred_mask.squeeze(0)
  pred_mask = pred_mask.unsqueeze(2)
  gt_mask = gt_mask.squeeze(0)
  gt_mask = gt_mask.unsqueeze(2)
  #print(img.shape)
  #img = img.reshape(img.shape[1],img.shape[2],img.shape[0])
  #print(np.array(img.cpu()))
  img = cv2.resize(img,dsize=(2000,2000))
  cv2_imshow(img)
  #cv2_imshow(np.array(pred_mask.cpu())*255)
  #cv2_imshow(np.array(gt_mask.cpu())*255)
  print("----------------------------------")

In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the valid set
'''
set_name = 'val'
my_data_list = DatasetCatalog.get("data_detection_{}".format('val'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, set_name)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the training set
'''
set_name = 'train'
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, set_name)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)


'''
# Writing the predictions of the test set
'''
set_name = 'test'
my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, set_name)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
for d in ["train", "test", "val"]:
  DatasetCatalog.register("data_mask_" + d, lambda d=d: get_detection_data(d))
  MetadataCatalog.get("data_mask_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("data_mask_train")

### Network

In [ ]:
cfgm = get_cfg()
cfgm.OUTPUT_DIR = "{}/outputm/".format(BASE_DIR)
cfgm.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfgm.DATASETS.TRAIN = ("data_mask_train",)
cfgm.DATASETS.TEST = ("data_mask_val",)
cfgm.DATALOADER.NUM_WORKERS = 2
cfgm.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfgm.SOLVER.IMS_PER_BATCH = 2 # 2 = 1 58%
cfgm.SOLVER.BASE_LR = 0.025  # pick a good LR
cfgm.SOLVER.MAX_ITER = 500
cfgm.SOLVER.STEPS = [200,]#[300,]        # decay learning rate
cfgm.SOLVER.GAMMA = 0.1
cfgm.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfgm.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfgm.MODEL.RETINANET.NUM_CLASSES = 1
cfgm.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 128
#cfg.INPUT.MIN_SIZE_TRAIN = 1000
#cfg.INPUT.MIN_SIZE_TEST = 1000
cfgm.INPUT.MAX_SIZE_TRAIN = 3000 # 8000=6000 = 4000 58%
cfgm.INPUT.MAX_SIZE_TEST = 3000

#cfgm.TEST.DETECTIONS_PER_IMAGE = 1000

### Training

In [ ]:
os.makedirs(cfgm.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfgm) 
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [ ]:
cfgm.MODEL.WEIGHTS = os.path.join(cfgm.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfgm.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
cfgm.MODEL.RETINANET.SCORE_THRESH_TEST = 0.6 #for RETINA
predictor = DefaultPredictor(cfgm)

dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
evaluator = COCOEvaluator("data_mask_val", output_dir = cfgm.OUTPUT_DIR)
train_loader = build_detection_test_loader(cfgm, "data_mask_val")
print(inference_on_dataset(predictor.model, train_loader, evaluator))